In [7]:
import numpy as np
from scipy.stats import norm

# Parameters
N = 100  # Number of users
M = 100  # Number of items
K = 10   # Dimension of embeddings
num_simulations = 20  # Number of simulations
search_cost = 0.5  # Search cost

# Function to calculate reservation utility zij
def calculate_reservation_utility(delta, cost):
    m = norm.ppf(1 - cost)  # Inverse CDF (quantile function) for standard normal
    zij = delta + m
    return zij

# Initialize total recommendations received by each item
total_recommendations_received = np.zeros(M, dtype=int)

# Run the simulation num_simulations times
for sim in range(num_simulations):
    # Initialize embeddings
    u = np.random.normal(0, 1, (N, K))
    v = np.random.normal(0, 1, (M, K))

    # Simulate delta_ij and e_ij
    delta = np.random.normal(0, 1, (N, M))
    e = np.random.normal(0, 0.1, (N, M))

    # Utility matrix U
    U = delta + e

    # Calculate reservation utility zij for all items and users
    z = calculate_reservation_utility(delta, search_cost)

    # Simulation of interactions and training data preparation
    interactions = []
    recommendations_received = np.zeros(M, dtype=int)

    for i in range(N):
        # Recommend the item that maximizes dot(u_i, v_j)
        scores = np.dot(u[i], v.T)
        recommended_item = np.argmax(scores)

        # Count the recommendation for the item
        recommendations_received[recommended_item] += 1

        # User's initial search is always the recommended item
        searched_items = [recommended_item]
        true_utilities = [U[i, recommended_item]]

        # Continue searching other items based on delta and search cost
        remaining_items = np.argsort(-delta[i, :])  # Sort items by delta in decreasing order
        for item in remaining_items:
            if item == recommended_item or len(searched_items) >= 10:
                continue
            if max(true_utilities) >= z[i, item]:
                break
            searched_items.append(item)
            true_utilities.append(U[i, item])

        # User selects item with max utility in the consideration set
        chosen_item = searched_items[np.argmax(true_utilities)]

        # Collect interaction data
        interactions.append((i, chosen_item, 1))  # User i chose item chosen_item

    # Accumulate the recommendations received for each item
    total_recommendations_received += recommendations_received

    # Print the recommendations received in the current simulation
    print(f"Recommendations received in simulation {sim + 1}: {recommendations_received}")

# Print the total recommendations received after all simulations
print("Total number of recommendations received by each item across all simulations:", total_recommendations_received)


Recommendations received in simulation 1: [0 0 0 0 2 1 0 0 0 0 0 1 8 0 1 2 5 0 0 0 6 0 0 1 2 0 0 1 5 1 0 1 0 1 0 0 2
 0 1 8 1 0 0 0 0 1 0 1 1 2 1 5 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 2 1
 0 0 1 0 2 3 0 2 1 0 0 0 0 1 0 1 0 2 0 2 3 1 1 1 3 7]
Recommendations received in simulation 2: [ 1  1  1  0  0  0  1  1  2  3  0  1  7  0  0  1  0  0  2  1  0  0  0  2
  0  0  1  2  0  5  0  0  0  0  0  1  0  1  0  1  0  0  0  0  4  1  0  1
  1  5  0  0  0  0  0  2  0  2  0  0  0  4  2  0  0 12  1  1  0  1  2  0
  0  0  0  0  0  0  2  3  2  0  0  1  3  0  1  0  0  0  0  2  6  3  0  0
  2  0  0  1]
Recommendations received in simulation 3: [4 0 0 4 0 2 0 1 0 5 0 2 0 0 5 0 1 0 0 1 0 0 3 0 0 0 0 0 0 0 0 1 1 0 0 3 2
 0 1 1 0 2 4 0 2 0 0 0 0 1 6 4 0 2 0 0 0 4 0 0 0 2 1 0 0 3 0 0 1 0 0 1 0 0
 5 1 2 1 0 2 0 2 0 0 0 0 0 0 0 1 0 5 3 1 3 0 1 1 1 1]
Recommendations received in simulation 4: [2 0 1 0 3 2 0 0 0 0 0 0 0 0 3 0 4 0 5 0 0 1 0 0 4 1 0 0 0 0 2 2 6 2 0 0 2
 1 0 1 2 1 2 0 2 0 0 0 4 0 9 0 0 0 0 0 2 1 